# Predicting Performing Arts Attendance with Machine Learning

## - SHAP Analysis

August 8, 2025

---

In [ ]:
import os
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_curve 
from sklearn.metrics import auc, roc_auc_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.utils.class_weight import compute_sample_weight

from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import ColumnTransformer

import shap
shap.initjs()  # load JS for visualization in Jupyter

## Load & clean the data

In [ ]:
%run 'code_01_data_cleaning.ipynb'

In [ ]:
fig_class

In [ ]:
df.shape

In [ ]:
y_vars = ['ATTEND']
df_Y = df[y_vars]
df_X = df.drop(columns=y_vars)

In [ ]:
df_Y.head()

In [ ]:
df_X.head()

## Seeds

In [ ]:
import random
import hashlib

def md5_hash(input_string):
    """Generates an MD5 hash from a given string.
    Args:
    input_string: The string to hash.
    Returns:
    The MD5 hash as a hexadecimal string.
    """
    md5_hasher = hashlib.md5()
    md5_hasher.update(input_string.encode('utf-8'))
    return md5_hasher.hexdigest()

In [ ]:
input_string = "performingartsattendance"
hashed_value = md5_hash(input_string)
print(f"The MD5 hash of '{input_string}' is: {hashed_value}")

# Convert the hexadecimal hash to an integer
try:
    number = int(hashed_value, 16)
    print(f"The integer representation of the hash is: {number}")
except ValueError:
    print("Invalid hexadecimal string")

# Set the seed value
random.seed(number)

print(f"Initial seed number: {number}")

# Generate a list of random numbers
n_seeds = 5
random.seed(number)
a = 0
b = 2**31-1
seeds = [random.randint(a, b) for _ in range(n_seeds)]

# Print the list
print("Seed", seeds)

## Set split

In [ ]:
def set_split(seed, df_X, df_y, test_size=0.2):
    return train_test_split(df_X, df_y, test_size=test_size, random_state=seed)

## Preprocessing

In [ ]:
categorical_vars = ['REGION', 'STATEFIP', 'METRO', 
                    'SEX', 'RACE', 'HISPAN', 'VETSTAT', 'YRIMMIG', 'MARST',
                    'EMPSTAT', 'CLASSWKR',
                    'EDUC99',
                    'SCHLCOLL', 'PROFCERT',
                    'DIFFHEAR', 'DIFFEYE', 'DIFFREM',
                    'DIFFPHYS', 'DIFFMOB', 'DIFFANY']
numerical_vars = [col for col in df_X.columns if col not in categorical_vars]

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(drop='first', handle_unknown='ignore'))
])
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median'))
])

preprocessor = ColumnTransformer([
    ('cat', cat_pipeline, categorical_vars),
    ('num', num_pipeline, numerical_vars)
])

## Evaluation

In [ ]:
# clf: trained
def evaluate_model(seed, clf, X_test_transformed, y_test):
    # Predict
    y_pred = clf.predict(X_test_transformed)
    y_prob = clf.predict_proba(X_test_transformed)[:, 1]

    return {
        'seed': seed,
        'f1': f1_score(y_test, y_pred),
        'auc': roc_auc_score(y_test, y_prob),
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred),
        'recall': recall_score(y_test, y_pred),
        'roc_curve': roc_curve(y_test, y_prob),
        'report': classification_report(y_test, y_pred),
        'y_test': y_test,
        'y_pred': y_pred,
        'y_prob': y_prob,
        'clf': clf
    }

## GB SHAP Analysis

In [ ]:
ycol = 'ATTEND'
bestmodel = 'GB'
bestcv = '10'

In [ ]:
# Collect SHAP Explanation objects from each seed
shap_values_array = []
base_value_array = []
X_transformed_display_list = []

# Fit the preprocessor once
preprocessor.fit(df_X)

# Utility to extract transformed feature names
def get_feature_names(preprocessor):
    feature_names = []
    for name, transformer, cols in preprocessor.transformers_:
        if transformer == 'drop':
            continue
        elif transformer == 'passthrough':
            feature_names.extend(cols)
        elif hasattr(transformer, 'get_feature_names_out'):
            names = transformer.get_feature_names_out(cols)
            feature_names.extend(names)
        else:
            raise ValueError(f"Unhandled transformer type: {name}")
    return feature_names

transformed_feature_names = get_feature_names(preprocessor)


for seed in seeds[:2]:
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(df_X, df_Y['ATTEND'], test_size=0.2, random_state=seed)

    # Transform using preprocessor
    X_train_transformed = preprocessor.transform(X_train).toarray()
    X_test_transformed = preprocessor.transform(X_test).toarray()

    # Fit final model
    final_model = joblib.load(f"models/{ycol}_{bestmodel}_cv{bestcv}_seed{seed}.pkl")
    final_model.fit(X_train_transformed, y_train)

    # SHAP explanation
    X_sample = X_test_transformed[:1000]
    explainer = shap.TreeExplainer(final_model)
    shap_values = explainer(X_sample)
    
    # Collect
    shap_values_array.append(shap_values.values)
    base_value_array.append(shap_values.base_values)
    
    X_df_sample = pd.DataFrame(X_sample, columns=transformed_feature_names).astype(float)
    X_transformed_display_list.append(X_df_sample)
    
# Combine results
shap_values_concat = np.vstack(shap_values_array)               # shape (n_total, n_features)
base_values_concat = np.hstack(base_value_array)               # shape (n_total,)
X_display_concat = pd.concat(X_transformed_display_list, ignore_index=True)

In [ ]:
# Visualize (first 100 for force plot)
shap.initjs()
shap.force_plot(
    base_values_concat[:1000],
    shap_values_concat[:1000],
    X_display_concat.iloc[:1000],
    feature_names=transformed_feature_names
)

In [ ]:
shap.summary_plot(
    shap_values_concat,
    X_display_concat,
    feature_names=transformed_feature_names,
#     max_display=10
)

In [ ]:
# Compute mean absolute SHAP value for each feature
mean_abs_shap = np.abs(shap_values_concat).mean(axis=0)

# Build DataFrame
shap_importance_df = pd.DataFrame({
    'Feature': transformed_feature_names,
    'MeanAbsSHAP': mean_abs_shap
}).sort_values(by='MeanAbsSHAP', ascending=False).reset_index(drop=True)

# Display top 20
shap_importance_df.head(20)

In [ ]:
# Group OneHot columns
shap_df = pd.DataFrame(
    shap_values_concat,
    columns=transformed_feature_names
)

grouped_shap_df = pd.DataFrame()
for var in categorical_vars:
    cols = [c for c in shap_df.columns if c.startswith(var + '_')]
    if cols:
        grouped_shap_df[var] = shap_df[cols].abs().sum(axis=1)
for var in numerical_vars:
    grouped_shap_df[var] = shap_df[var].abs()


mean_abs_importance = grouped_shap_df.mean().sort_values(ascending=False)
mean_abs_importance

In [ ]:
# Individual instance
i = 0

shap.force_plot(
    base_values_concat[i],
    shap_values_concat[i],
    X_display_concat.iloc[i],
    feature_names=transformed_feature_names,
    matplotlib=True
)

In [ ]:
# Create Explanation object if needed
single_explanation = shap.Explanation(
    values=shap_values_concat[i],
    base_values=base_values_concat[i],
    data=X_display_concat.iloc[i].values,
    feature_names=transformed_feature_names
)

# Waterfall plot
shap.plots.waterfall(single_explanation)

In [ ]:
i = 42

shap.force_plot(
    base_values_concat[i],
    shap_values_concat[i],
    X_display_concat.iloc[i],
    feature_names=transformed_feature_names,
    matplotlib=True
)

In [ ]:
# Create Explanation object if needed
single_explanation = shap.Explanation(
    values=shap_values_concat[i],
    base_values=base_values_concat[i],
    data=X_display_concat.iloc[i].values,
    feature_names=transformed_feature_names
)

# Waterfall plot
shap.plots.waterfall(single_explanation)